In [2]:
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
import torch

## Get the Data and Labels

In [3]:
num_testsub = 32
num_classes = 4
num_channel = 40
num_datapoints = 8064
num_trials = 40
sampling_rate = 128 # 128Hz as given in the data

def loadfiles_normalized():
    data_dict = {}
    print("Loading files into data_dict .................")
    for i in range(num_testsub):
                    if i < 10:
                        name = '%0*d' % (2,i+1)
                    else:
                        name = i+1
                    fname = 'data/data_prepared/data_norm_bhat/noramlized_datasub'+str(name) +'.npy'
                    data_dict["sub%s" %name] = np.load(fname)    
    print("Loaded!!!!!") 
    return data_dict

In [4]:
data_dict = loadfiles_normalized()

Loading files into data_dict .................
Loaded!!!!!


In [5]:
valence = np.load('data/data_prepared/labels/label_class_0.npy')
arousal = np.load('data/data_prepared/labels/label_class_1.npy')
dominance = np.load('data/data_prepared/labels/label_class_2.npy')
liking = np.load('data/data_prepared/labels/label_class_3.npy')

In [6]:
valence.shape

(1280,)

In [7]:
data_dict['sub01'].shape

(40, 40, 99)

## Create train test split

### Data

In [70]:
# training with 22 participants each has 40 videos = 880
train = np.zeros((0,40,99))
train[:] = np.nan
#train1 = 
test = np.zeros((0,40,99))
test[:] = np.nan

participation_counter = 0
for sub in data_dict.keys():
    if participation_counter < 22:
        train = np.concatenate((train, data_dict[sub]), axis=0)
    else:
        test = np.concatenate((test, data_dict[sub]), axis=0)
    participation_counter = participation_counter + 1

In [9]:
np.isnan(train).any()

False

In [10]:
np.isnan(test).any()

False

In [11]:
train.shape

(880, 40, 99)

In [12]:
test.shape

(400, 40, 99)

### Get generated Data

In [71]:
new_data = torch.load("data/psd_427")
new_data = new_data.numpy()

In [72]:
new_data.shape

(427, 3, 40, 104)

In [73]:
X_argument = np.zeros((880,40,99))
X_argument[:] = np.nan
counter = 0
for i in range(427):
    for j in range(3):
        if counter == 880:
            break
        else:
            temp = new_data[i][j][:,:99]
            X_argument[counter] = new_data[i][j][:,:99]
            counter = counter +1

In [74]:
np.isnan(X_argument).any()

False

In [75]:
# 2x
x2 = np.concatenate([X_argument, train], 0) 
print(x2.shape)

(1760, 40, 99)


----------------

In [76]:
arrays = []
for i in range(10):
    arrays.append(torch.load("data/sampled_40-104_43_"+str(i)))
generated_data = np.concatenate(arrays, axis=0)

In [77]:
X_argument2 = np.zeros((880,40,99))
X_argument2[:] = np.nan
counter = 0
for i in range(430):
    for j in range(3):
        if counter == 880:
            break
        else:
            temp = generated_data[i][j][:,:99]
            X_argument2[counter] = generated_data[i][j][:,:99]
            counter = counter +1

In [78]:
# 2x
x2 = np.concatenate([X_argument2, train], 0) 
print(x2.shape)

(1760, 40, 99)


In [81]:
np.array_equal(X_argument, train)

False

In [79]:
np.array_equal(X_argument2, train)

False

In [80]:
np.array_equal(X_argument, X_argument2)

False

### Labels

In [18]:
y_train_va = valence[: 880]
y_train_ar = arousal[: 880]
y_train_do = dominance[: 880]
y_train_li = liking[: 880]

y_test_va = valence[880: ]
y_test_ar = arousal[880: ]
y_test_do = dominance[880: ]
y_test_li = liking[880: ]

In [19]:
# Augmenting the labels 
# 2x
y_train_va2 = np.hstack((y_train_va,y_train_va))
y_train_ar2 = np.hstack((y_train_ar,y_train_ar))
y_train_do2 = np.hstack((y_train_do,y_train_do))
y_train_li2 = np.hstack((y_train_li,y_train_li))

In [20]:
print(y_train_va.shape)
print(y_train_ar.shape)
print(y_train_do.shape)
print(y_train_li.shape)
print(y_test_va.shape)
print(y_test_ar.shape)
print(y_test_do.shape)
print(y_test_li.shape)

(880,)
(880,)
(880,)
(880,)
(400,)
(400,)
(400,)
(400,)


## KNN

In [21]:
train.reshape(-1, 40*99).shape

(880, 3960)

In [31]:
# Define KNN based model for actual/original dataset
def knnmodel(xtrain,ytrain,xtest,ytest,kval):
  random_state=100
  # print("xtrain.shape,ytrain.shape",xtrain.shape,ytrain.shape)
  # print("xtest.shape,ytest.shape",xtrain.shape,ytrain.shape)
  # Define the KNN model here

  model = KNeighborsClassifier(n_neighbors=kval) 
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  # print(classification_report(ytest, y_pred))
  # print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [32]:
# Define KNN based model for augmented data
def knnmodel2(xtrain,ytrain,xtest,ytest,kval):
  
  # print("xtrain.shape,ytrain.shape",xtrain.shape,ytrain.shape)
  # print("xtest.shape,ytest.shape",xtest.shape,ytest.shape)

  # Define the KNN model here

  model = KNeighborsClassifier(n_neighbors=kval)
  model.fit(xtrain, ytrain)
  y_pred = model.predict(xtest)

  # print("Accuracy score for fold",fold_no)
  acc = accuracy_score(ytest, y_pred)*100
  acc = round(acc, 4)
  print("Accuracy",acc)
  #print(classification_report(ytest, y_pred))
  #print(confusion_matrix(ytest,y_pred))
  # return acc,xtrain, xtest, ytrain, ytest
  return acc

In [89]:
trainlabel_list = [y_train_va, y_train_ar, y_train_do, y_train_li]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []


for k in range(1,60):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel(train.reshape(-1, 40*99),traain,test.reshape(-1, 40*99),tesst,kval = k)
    scorelist.append((sc,k))
  
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4

#for i,name in zip(scorelist,names):
#    print ("Accuracy of ",name ,i)

K value = 1
######################################### Valence #########################################
Accuracy 50.5
######################################### Arousal #########################################
Accuracy 48.0
######################################### Dominance #########################################
Accuracy 53.5
######################################### Liking #########################################
Accuracy 50.0
K value = 2
######################################### Valence #########################################
Accuracy 43.25
######################################### Arousal #########################################
Accuracy 47.5
######################################### Dominance #########################################
Accuracy 42.5
######################################### Liking #########################################
Accuracy 46.0
K value = 3
######################################### Valence #########################################
Accuracy 50.5
#######

Accuracy 54.25
######################################### Liking #########################################
Accuracy 59.0
K value = 20
######################################### Valence #########################################
Accuracy 50.0
######################################### Arousal #########################################
Accuracy 56.75
######################################### Dominance #########################################
Accuracy 52.0
######################################### Liking #########################################
Accuracy 57.75
K value = 21
######################################### Valence #########################################
Accuracy 52.0
######################################### Arousal #########################################
Accuracy 53.75
######################################### Dominance #########################################
Accuracy 56.25
######################################### Liking #########################################
Accuracy 58.5


Accuracy 52.75
######################################### Arousal #########################################
Accuracy 52.75
######################################### Dominance #########################################
Accuracy 56.25
######################################### Liking #########################################
Accuracy 62.0
K value = 40
######################################### Valence #########################################
Accuracy 52.0
######################################### Arousal #########################################
Accuracy 51.5
######################################### Dominance #########################################
Accuracy 55.0
######################################### Liking #########################################
Accuracy 61.5
K value = 41
######################################### Valence #########################################
Accuracy 51.75
######################################### Arousal #########################################
Accuracy 52.0


Accuracy 54.75
######################################### Liking #########################################
Accuracy 62.25
K value = 59
######################################### Valence #########################################
Accuracy 51.25
######################################### Arousal #########################################
Accuracy 52.75
######################################### Dominance #########################################
Accuracy 58.25
######################################### Liking #########################################
Accuracy 62.5


In [24]:
len(scorelist)

196

In [90]:
print(max_va)
print(max_ar)
print(max_do)
print(max_li)

(54.5, 55)
(56.75, 20)
(58.75, 7)
(63.0, 49)


### 2X Data

In [82]:
x2.reshape(-1, 40*99).shape

(1760, 3960)

In [91]:
trainlabel_list = [y_train_va2, y_train_ar2, y_train_do2, y_train_li2]
testlabel_list = [y_test_va, y_test_ar, y_test_do, y_test_li]
names = ["Valence","Arousal","Dominance","Liking"]
scorelist = []
scores = []

for k in range(1,60):
  print("K value =",k)
  for traain,tesst,name in zip(trainlabel_list,testlabel_list,names):
    print("#########################################",name,"#########################################")
    sc = knnmodel2(x2.reshape(-1, 40*99),traain,test.reshape(-1, 40*99),tesst,kval = k)
    scorelist.append((sc,k))
    scores.append(sc)
  
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4


K value = 1
######################################### Valence #########################################
Accuracy 50.5
######################################### Arousal #########################################
Accuracy 48.0
######################################### Dominance #########################################
Accuracy 53.5
######################################### Liking #########################################
Accuracy 50.0
K value = 2
######################################### Valence #########################################
Accuracy 43.25
######################################### Arousal #########################################
Accuracy 47.5
######################################### Dominance #########################################
Accuracy 42.5
######################################### Liking #########################################
Accuracy 46.0
K value = 3
######################################### Valence #########################################
Accuracy 50.5
#######

Accuracy 54.25
######################################### Liking #########################################
Accuracy 59.0
K value = 20
######################################### Valence #########################################
Accuracy 50.0
######################################### Arousal #########################################
Accuracy 56.75
######################################### Dominance #########################################
Accuracy 52.0
######################################### Liking #########################################
Accuracy 57.75
K value = 21
######################################### Valence #########################################
Accuracy 52.0
######################################### Arousal #########################################
Accuracy 53.75
######################################### Dominance #########################################
Accuracy 56.25
######################################### Liking #########################################
Accuracy 58.5


Accuracy 53.75
######################################### Dominance #########################################
Accuracy 55.0
######################################### Liking #########################################
Accuracy 60.25
K value = 39
######################################### Valence #########################################
Accuracy 52.75
######################################### Arousal #########################################
Accuracy 52.75
######################################### Dominance #########################################
Accuracy 56.25
######################################### Liking #########################################
Accuracy 62.0
K value = 40
######################################### Valence #########################################
Accuracy 52.0
######################################### Arousal #########################################
Accuracy 51.5
######################################### Dominance #########################################
Accuracy 55

Accuracy 51.75
######################################### Arousal #########################################
Accuracy 54.0
######################################### Dominance #########################################
Accuracy 57.25
######################################### Liking #########################################
Accuracy 62.5
K value = 58
######################################### Valence #########################################
Accuracy 51.0
######################################### Arousal #########################################
Accuracy 53.25
######################################### Dominance #########################################
Accuracy 54.75
######################################### Liking #########################################
Accuracy 62.25
K value = 59
######################################### Valence #########################################
Accuracy 51.25
######################################### Arousal #########################################
Accuracy 52.

In [36]:
len(scorelist)

196

In [92]:
print(max_va)
print(max_ar)
print(max_do)
print(max_li)

(54.5, 55)
(56.75, 20)
(58.75, 7)
(63.0, 49)


In [93]:
np.max(scores)

63.0

In [ ]:
scorelist = [(15,0),(8,0),(88,0),(40,9),
             (5,6),(67,9),(7,6),(23,0),
             (12,2),(3,1),(5,3),(78,8)]
max_va = (0,0)
max_ar = (0,0)
max_do = (0,0)
max_li = (0,0)
j = 0
for i in range(int(len(scorelist)/4)):
    if scorelist[j][0] > max_va[0]:
        max_va = scorelist[j]
    if scorelist[j+1][0] > max_ar[0]:
        max_ar = scorelist[j+1]
    if scorelist[j+2][0] > max_do[0]:
        max_do = scorelist[j+2]
    if scorelist[j+3][0] > max_li[0]:
        max_li = scorelist[j+3]
    j = j+4